In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

mlflow ui --backend-store-uri sqlite:///mlflow.db

In [3]:
import os

In [23]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops/02-experiment_tracking/mlruns/1', creation_time=1749562952052, experiment_id='1', last_update_time=1749562952052, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [24]:
mlflow.end_run()

In [5]:
mlflow.get_artifact_uri()

'/workspaces/mlops/02-experiment_tracking/mlruns/1/4ce6d8aa93ed416ab2dfd884fbe57b10/artifacts'

In [4]:
!python -V

Python 3.13.2


In [7]:
os.getcwd()

'/workspaces/mlops/02-experiment_tracking'

In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
ls ..

01-intro/  02-experiment_tracking/  LICENSE  README.md  data/  mlflow.db


In [10]:
!ls ../data

green_tripdata_2021-01.parquet	green_tripdata_2021-02.parquet


In [8]:
df_train = read_dataframe('../data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train.dtypes

VendorID                          int64
lpep_pickup_datetime     datetime64[us]
lpep_dropoff_datetime    datetime64[us]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                     object
DOLocationID                     object
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                        object
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
duration                        float64
dtype: object

In [11]:
df_train.iloc[0]

VendorID                                   2
lpep_pickup_datetime     2021-01-01 00:15:56
lpep_dropoff_datetime    2021-01-01 00:19:52
store_and_fwd_flag                         N
RatecodeID                               1.0
PULocationID                              43
DOLocationID                             151
passenger_count                          1.0
trip_distance                           1.01
fare_amount                              5.5
extra                                    0.5
mta_tax                                  0.5
tip_amount                               0.0
tolls_amount                             0.0
ehail_fee                               None
improvement_surcharge                    0.3
total_amount                             6.8
payment_type                             2.0
trip_type                                1.0
congestion_surcharge                     0.0
duration                            3.933333
Name: 0, dtype: object

In [12]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [13]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [14]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715208009878

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:

# mlflow.end_run()

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "sharaf")

    mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
  
    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [16]:
dv.feature_names_

['PU_DO=100_168',
 'PU_DO=100_180',
 'PU_DO=100_190',
 'PU_DO=100_225',
 'PU_DO=100_235',
 'PU_DO=100_42',
 'PU_DO=100_49',
 'PU_DO=100_69',
 'PU_DO=100_82',
 'PU_DO=100_89',
 'PU_DO=101_101',
 'PU_DO=101_13',
 'PU_DO=101_135',
 'PU_DO=101_139',
 'PU_DO=101_16',
 'PU_DO=101_175',
 'PU_DO=101_188',
 'PU_DO=101_19',
 'PU_DO=101_191',
 'PU_DO=101_196',
 'PU_DO=101_205',
 'PU_DO=101_218',
 'PU_DO=101_219',
 'PU_DO=101_226',
 'PU_DO=101_260',
 'PU_DO=101_265',
 'PU_DO=101_64',
 'PU_DO=101_7',
 'PU_DO=101_75',
 'PU_DO=101_86',
 'PU_DO=101_95',
 'PU_DO=102_102',
 'PU_DO=102_117',
 'PU_DO=102_121',
 'PU_DO=102_130',
 'PU_DO=102_131',
 'PU_DO=102_134',
 'PU_DO=102_139',
 'PU_DO=102_145',
 'PU_DO=102_16',
 'PU_DO=102_160',
 'PU_DO=102_164',
 'PU_DO=102_182',
 'PU_DO=102_196',
 'PU_DO=102_197',
 'PU_DO=102_198',
 'PU_DO=102_205',
 'PU_DO=102_21',
 'PU_DO=102_219',
 'PU_DO=102_235',
 'PU_DO=102_236',
 'PU_DO=102_242',
 'PU_DO=102_249',
 'PU_DO=102_258',
 'PU_DO=102_263',
 'PU_DO=102_28',
 'PU_DO=1

In [17]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [19]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [20]:
train, valid

(<xgboost.core.DMatrix at 0x76cc90848cd0>,
 <xgboost.core.DMatrix at 0x76cca7b110f0>)

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
# search_space = {
#     'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
#     'learning_rate': hp.loguniform('learning_rate', -3, 0),
#     'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
#     'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
#     'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
#     'objective': 'reg:linear',
#     'seed': 42
# }

# best_result = fmin(
#     fn=objective,
#     space=search_space,
#     algo=tpe.suggest,
#     max_evals=50,
#     trials=Trials()
# )

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/miniconda3/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [19:48:42] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.40043                           
[1]	validation-rmse:7.19202                           
[2]	validation-rmse:6.83083                           
[3]	validation-rmse:6.70396                           
[4]	validation-rmse:6.65497                           
[5]	validation-rmse:6.62963                           
[6]	validation-rmse:6.61334                           
[7]	validation-rmse:6.59757                           
[8]	validation-rmse:6.59347                           
[9]	validation-rmse:6.58977                           
[10]	validation-rmse:6.58764                          
[11]	validation-rmse:6.58465                          
[12]	validation-rmse:6.58262                          
[13]	validation-rmse:6.57349                          
[14]	validation-rmse:6.57114                          
[15]	validation-rmse:6.57004                          
[16]	validation-rmse:6.56834                          
[17]	validation-rmse:6.56679                          
[18]	valid

/home/codespace/miniconda3/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [19:50:00] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.66341                                                      
[1]	validation-rmse:7.27266                                                      
[2]	validation-rmse:6.77562                                                      
[3]	validation-rmse:6.58587                                                      
[4]	validation-rmse:6.51014                                                      
[5]	validation-rmse:6.46716                                                      
[6]	validation-rmse:6.44463                                                      
[7]	validation-rmse:6.43295                                                      
[8]	validation-rmse:6.42797                                                      
[9]	validation-rmse:6.42215                                                      
[10]	validation-rmse:6.41860                                                     
[11]	validation-rmse:6.41338                                                     
[12]	validation-

...

[0]	validation-rmse:11.62465                                                       
[1]	validation-rmse:11.09206                                                       
[2]	validation-rmse:10.61154                                                       
[3]	validation-rmse:10.17798                                                       
[4]	validation-rmse:9.78755                                                        
[5]	validation-rmse:9.43800                                                        
[6]	validation-rmse:9.12294                                                        
[7]	validation-rmse:8.84209                                                        
[8]	validation-rmse:8.59031                                                        
[9]	validation-rmse:8.36754                                                        
[10]	validation-rmse:8.16813                                                       
[11]	validation-rmse:7.99160                                                

/home/codespace/miniconda3/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [21:56:40] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:8.52316                                                        
[1]	validation-rmse:7.19836                                                        
[2]	validation-rmse:6.75609                                                        
[3]	validation-rmse:6.59139                                                        
[4]	validation-rmse:6.52598                                                        
[5]	validation-rmse:6.49357                                                        
[6]	validation-rmse:6.47072                                                        
[7]	validation-rmse:6.45595                                                        
[8]	validation-rmse:6.45068                                                        
[9]	validation-rmse:6.44552                                                        
[10]	validation-rmse:6.44126                                                       
[11]	validation-rmse:6.43687                                                

/home/codespace/miniconda3/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [21:57:43] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:10.00826                                                       
[1]	validation-rmse:8.63930                                                        
[2]	validation-rmse:7.82120                                                        
[3]	validation-rmse:7.34420                                                        
[4]	validation-rmse:7.06595                                                        
[5]	validation-rmse:6.90151                                                        
[6]	validation-rmse:6.80365                                                        
[7]	validation-rmse:6.74194                                                        
[8]	validation-rmse:6.70189                                                        
[9]	validation-rmse:6.67253                                                        
[10]	validation-rmse:6.65503                                                       
[11]	validation-rmse:6.64086                                                

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        

In [22]:
import os
os.getcwd()

'/workspaces/mlops/02-experiment_tracking'

In [6]:
!ls

duration_prediction.ipynb  mlflow.db  mlruns  models  requirements.txt


In [25]:
dv.get_feature_names_out()

array(['PU_DO=100_168', 'PU_DO=100_180', 'PU_DO=100_190', ...,
       'PU_DO=9_97', 'PU_DO=9_98', 'trip_distance'],
      shape=(13221,), dtype=object)

In [23]:
dv.get_feature_names_out()

array(['PU_DO=100_168', 'PU_DO=100_180', 'PU_DO=100_190', ...,
       'PU_DO=9_97', 'PU_DO=9_98', 'trip_distance'],
      shape=(13221,), dtype=object)

In [21]:
X_val[0]

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2 stored elements and shape (1, 13221)>

In [24]:
mlflow.xgboost.autolog(disable=True)

In [27]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    # mlflow.xgboost.log_model(booster, artifact_path="models_mlflow", registered_model_name="best_xgboost_model", input_example=X_val[:5], signature=dv.get_feature_names_out())
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow", registered_model_name="best_xgboost_model", input_example=X_val[0])

/home/codespace/miniconda3/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [18:50:08] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

/home/codespace/miniconda3/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [18:53:25] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
Successfully registered model 'best_xgboost_model'.
Created version '1' of model 'best_xgboost_model'.


In [25]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "../data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        

: 